# Demo notebook for Model Reader

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Delete the time files before running.
model = 'GAMERA_GM'
variables_requested = ['B_y','N_plasma','X','dV']  # 'B_0z' only in serial run
# change file path to where data is stored on your machine
#file_dir = 'D:/GAMERA_GM/SerialRun/'  # 2022-01-12
#file_dir = 'D:/GAMERA_GM/MPIRun/'  # 2022-01-12, 1858 date b/c of time value error in files.
file_dir = '/Users/lrastaet/Kamodo_data/GAMERA/Gerkin_Ai_070323_1/results/output_0/'
#file_dir = '/Users/lrastaet/Kamodo_data/GAMERA/Slava_Merkin_120423_1/results/output_0/'
from os.path import isfile
from os import remove
if isfile(file_dir+model+'_times.txt'):
    remove(file_dir+model+'_times.txt')
if isfile(file_dir+model+'_list.txt'):
    remove(file_dir+model+'_list.txt')

In [ ]:
# confirm MW functions work appropriately with model varnames dictionary
import kamodo_ccmc.flythrough.model_wrapper as MW
MW.Variable_Search('magnetic')

In [ ]:
MW.Variable_Search('magnetic', model)

In [ ]:
MW.Variable_Search('magnetic', model, file_dir)

In [ ]:
MW.Var_3D(model)

In [ ]:
MW.Var_units(model, variables_requested)

In [ ]:
# Confirm time method works with model reader
MW.File_Times(model, file_dir)

In [ ]:
# Confirm file list method works with model reader
MW.File_List(model, file_dir)

In [ ]:
# Check that time files creation works, that reader works for one variable,
# and that an unknown variable request does not break it.
reader = MW.Model_Reader(model)
kamodo_object = reader(file_dir, variables_requested=['Trash'])
kamodo_object

In [ ]:
# Check that reader works for one variable with an unknown variable
kamodo_object = reader(file_dir, variables_requested=['Trash', variables_requested[0]])
kamodo_object

In [ ]:
print(kamodo_object.B_y_ijk(time=24.5,X=10.,Y=1.,Z=10.))

In [ ]:
# Test model reader with one variable from each coordinate system
kamodo_object = reader(file_dir, variables_requested=variables_requested)
kamodo_object

In [ ]:
# Check that reading the time files works and that reader works for one variable,
kamodo_object = reader(file_dir, variables_requested=variables_requested[:1])
kamodo_object

In [ ]:
kamodo_object.filedate

In [ ]:
# Confirm that interpolation works. 
from math import isnan
tpos,xpos,ypos,zpos = [21.,10.,15.,20.]
#xpos, ypos, zpos = [21.,10.,60. ,50.]
print(kamodo_object.B_y([21., xpos, ypos, zpos]))
if isnan(kamodo_object.B_y([21., xpos, ypos, zpos])[0]):
    raise AttributeError('Returned value is a NaN.')
else:
    print('Value is valid.')
print(kamodo_object.B_y_ijk(time=tpos, X=xpos, Y=ypos, Z=zpos))
if isnan(kamodo_object.B_y_ijk(time=tpos, X=xpos, Y=ypos, Z=zpos)):
    raise AttributeError('Returned value is a NaN.')
else:
    print('Value is valid.')
if not kamodo_object.B_y([tpos,xpos,ypos,zpos]) == kamodo_object.B_y_ijk(time=tpos, X=xpos, Y=ypos, Z=zpos):
    raise AttributeError('Values are not equal.')
else:
    print('Values are equal.')
print(kamodo_object.B_y_ijk(time=xpos, X=ypos).shape)

In [ ]:
# Check that reading the time files works, and that the reader works for all variables
kamodo_object = reader(file_dir)
kamodo_object

In [ ]:
# Get a list of all of the functionalized variables, both regular and gridded
var_list = list(MW.Variable_Search('', model, file_dir, return_dict=True).keys())
varijk_list = sorted(var_list + [item+'_ijk' for item in var_list])
varijk_list

In [ ]:
# Test coordinate range logic for all variables
MW.Coord_Range(kamodo_object, varijk_list)

In [ ]:
# Check that the kamodo object was built properly.
print(kamodo_object.B_z([tpos,xpos,ypos,zpos]))
if isnan(kamodo_object.B_z([tpos,xpos,ypos,zpos])[0]):
    raise AttributeError('Returned value is a NaN.')
else:
    print('Value is valid.')

In [ ]:
# Check that the reader works for the testing subset
kamodo_object = reader(file_dir, variables_requested=variables_requested)
kamodo_object

In [ ]:
# Confirm that the interpolator works for each testing variable and type
print(kamodo_object.B_y([tpos,xpos,ypos,zpos]))
if isnan(kamodo_object.B_y([tpos,xpos,ypos,zpos])[0]):
    raise AttributeError('Returned value is a NaN.')
else:
    print('Value is valid.')
print(kamodo_object.B_y_ijk(time=tpos, X=xpos, Y=ypos, Z=zpos))
if isnan(kamodo_object.B_y_ijk(time=tpos, X=xpos, Y=ypos, Z=zpos)):
    raise AttributeError('Returned value is a NaN.')
else:
    print('Value is valid.')
if not kamodo_object.B_y([tpos,xpos,ypos,zpos]) == kamodo_object.B_y_ijk(time=tpos, X=xpos, Y=ypos, Z=zpos):
    raise AttributeError('Values are not equal.')
else:
    print('Values are equal.')
print(kamodo_object.B_y_ijk(time=tpos, X=xpos).shape)
# X
## dV not yet supported
if isnan(kamodo_object.X([xpos,ypos,zpos])[0]):
    raise AttributeError('Returned value is a NaN.')
else:
    print('Value is valid.')
print(kamodo_object.X_ijk(X=xpos, Y=ypos, Z=zpos))
if isnan(kamodo_object.X_ijk(X=xpos, Y=ypos, Z=zpos)):
    raise AttributeError('Returned value is a NaN.')
else:
    print('Value is valid.')
if not kamodo_object.X([xpos,ypos,zpos]) == kamodo_object.X_ijk(X=xpos, Y=ypos, Z=zpos):
    raise AttributeError('Values are not equal.')
else:
    print('Values are equal.')
print(kamodo_object.X_ijk(X=10).shape)

In [ ]:
# Generate a plot for validation
from kamodo_ccmc.tools.plotfunctions import toColor
toColor(kamodo_object.plot('B_y_ijk', plot_partial={'B_y_ijk': {'time': tpos, 'Z': zpos/2}}),
        colorscale="Viridis")

In [ ]:
kamodo_object.B_y([tpos,-190.,20.,zpos])

In [ ]:
# Generate a plot for validation
toColor(kamodo_object.plot('B_y_ijk', plot_partial={'B_y_ijk': {'X': xpos, 'Y': ypos}}),
        colorscale="Viridis")

In [ ]:
# Generate a plot for validation
toColor(kamodo_object.plot('dV_ijk', plot_partial={'dV_ijk': {'Y': 0.}}),
        colorscale="Viridis")

In [ ]:
# Test that more than one variable works through the flythrough
from kamodo_ccmc.flythrough import SatelliteFlythrough as SF
import datetime as dt
# valid time range:
# 2023-04-18  Time: 14:00:00
# 2023-04-19  Time: 02:00:00
start_utcts = dt.datetime(2023, 4, 18, 22, 5).replace(tzinfo=dt.timezone.utc).timestamp()
end_utcts = dt.datetime(2023, 4, 19, 1, 50).replace(tzinfo=dt.timezone.utc).timestamp()-1
results = SF.ModelFlythrough(model, file_dir, [variables_requested[0]], [start_utcts, end_utcts], 
                             [0., 10.], [10., -10.], [0., 0.], 'SM-car')
results[variables_requested[0]]

In [ ]:
# Test that one variable works through the flythrough
results = SF.ModelFlythrough(model, file_dir, [variables_requested[0]], [start_utcts], [0.], [10.],
                             [40.], 'GSM-car')
results[variables_requested[0]]